In [ ]:
import base64
import os
from mistralai import Mistral
import dotenv

# Load environment variables from a .env file if present
dotenv.load_dotenv()


# 1. Setup the client
# Ensure MISTRAL_API_KEY is set in your environment variables
api_key = os.environ.get("MISTRAL_API_KEY")

if not api_key:
    raise ValueError("Please set your MISTRAL_API_KEY environment variable.")

client = Mistral(api_key=api_key)

# 2. Define the file path
file_name = "contrat-bail-meuble.pdf"

# 3. Function to encode the local PDF to base64
def encode_pdf(pdf_path):
    try:
        with open(pdf_path, "rb") as pdf_file:
            return base64.b64encode(pdf_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file '{pdf_path}' was not found.")
        exit()

# 4. Process the document
print(f"Processing {file_name}...")
base64_pdf = encode_pdf(file_name)

ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "document_url",
        "document_url": f"data:application/pdf;base64,{base64_pdf}"
    },
    include_image_base64=False # Set to True if you also need image extraction
)

# 5. Output the text (Markdown)
# The response contains a list of pages, each with a 'markdown' attribute
for i, page in enumerate(ocr_response.pages):
    print(f"\n--- Page {i+1} ---\n")
    print(page.markdown)

In [ ]:
from openai import OpenAI
import os
import dotenv

# Load environment variables from a .env file if present
dotenv.load_dotenv()

# Ensure SCW_SECRET_KEY is set in your environment variables
api_key = os.environ.get("SCW_SECRET_KEY")

client = OpenAI(
    base_url = "https://313aa39a-7977-4e43-b624-72142f9c351e.ifr.fr-par.scaleway.com/v1",
    api_key = api_key
)


response = client.chat.completions.create(

  model="mistralai/Devstral-2-123B-Instruct-2512",
  messages=[
        { "role": "system", "content": "You are a helpful assistant" },
        { "role": "user", "content": "" },
  ],
  max_tokens=512,
  temperature=0.7,
  top_p=0.8,
  presence_penalty=0,
  stream=True,
  response_format={ "type": "text" }
)


for chunk in response:
  if chunk.choices and chunk.choices[0].delta.content:
   print(chunk.choices[0].delta.content, end="", flush=True)